In [2]:
#필요 패키지 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy==1.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp311-cp311-linux_x86_64.whl size=679486 sha256=c4893cee98c6fe8545c547722f822037bffa2a29a4ea27300fa851a5dad093ce
  Stored in directory: /root/.cache/pip/wheels/29/64/e3/047328bfc4a551696adc4a82dba3c980a1dad62198b8a3bb83
Successfully built gluonnlp
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━

In [4]:
#kobert
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-8jdrc4jf/kobert-tokenizer_3cd0c1dd6247426ca976191e918a5f10
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-8jdrc4jf/kobert-tokenizer_3cd0c1dd6247426ca976191e918a5f10
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=0ea36da73a595ed9849b4f617029768cc11e9bab4795e1c860a64df2b5d7c508
  Stored in directory: /tmp/pip-ephem-wheel-cache-v91t_1b1/wheels/b7/95/7c/3f38b039ae9422cc4a51702b37e352d0585442cbe60285f1f2
Successfully built kobert_tokenizer


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [5]:
#  Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 설정
device = torch.device("cuda:0")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [9]:
import pandas as pd
ori_t = pd.read_excel('/content/drive/MyDrive/sentimental_chat_Training.xlsx')
ori_v = pd.read_excel('/content/drive/MyDrive/sentimental_chat_Validation.xlsx')

In [10]:
ori_t.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [11]:
ori_t = ori_t.fillna("")
ori_v = ori_v.fillna("")

In [12]:
ori_t["사람문장"] = ori_t["사람문장1"].astype(str)+ori_t["사람문장2"].astype(str)+ori_t["사람문장3"].astype(str)
ori_v["사람문장"] = ori_v["사람문장1"].astype(str)+ori_v["사람문장2"].astype(str)+ori_v["사람문장3"].astype(str)

In [13]:
df_concat = pd.concat([ori_t, ori_v])
df_concat.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3,사람문장
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,,,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,,,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,,,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,,,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,,,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...


In [14]:
chatbot = df_concat[["사람문장", "감정_대분류"]]

In [15]:
chatbot = chatbot.rename({"감정_대분류": "Emotion"}, axis=1)
chatbot = chatbot.rename({"사람문장": "Sentence"}, axis=1)

In [16]:
chatbot["Emotion"] = chatbot["Emotion"].apply(lambda x:x.strip())

In [17]:
chatbot["Emotion"].value_counts()

,count
Emotion,
불안,10433
분노,10417
상처,10150
슬픔,10128
당황,9804
기쁨,7339


In [18]:
chatbot.head()

,Sentence,Emotion
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,분노


In [19]:
device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [20]:
# 6개의 감정 class -> 숫자
chatbot.loc[(chatbot['Emotion']=="불안"),'Emotion']=0 # 불안 => 0
chatbot.loc[(chatbot['Emotion']=="분노"),'Emotion']=1 # 분노 => 1
chatbot.loc[(chatbot['Emotion']=="상처"),'Emotion']=2 # 상처 => 2
chatbot.loc[(chatbot['Emotion']=="슬픔"),'Emotion']=3 # 슬픔 => 3
chatbot.loc[(chatbot['Emotion']=="당황"),'Emotion']=4 # 당황 => 4
chatbot.loc[(chatbot['Emotion']=="기쁨"),'Emotion']=5 # 기쁨 => 5

data_list = []
for q, label in zip(chatbot['Sentence'], chatbot['Emotion']):
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

print(data)
print(data_list[:10])

['친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네.맞아. 하지만 그렇다고 아무나하고 결혼할 수도 없잖아.', '4']
[['일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.', '1'], ['이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.', '1'], ['회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. 잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대하는 게 덜 스트레스받을 것 같아.', '1'], ['직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.', '1'], ['얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!', '1'], ['직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.직장 상사한테 자주 지적을 받아. 그럴 때마다 이 업무는 나랑 맞지 않는 거 같이 느껴져.', '1'], ['성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.부모님께 좀 더 시간을 달라고 해야겠어. 진로 테스트를 받아볼까 생각 중이야.', '1'], ['퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.더 좋은 회사가 기다리고 있을지도 모르는데 쉬고 있기가 좀 그래.', '5'], ['졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.응. 느긋함을 느끼는 내가 좀 싫었는데 걱정을 많이 하는 것보다는 낫다는 생각도 들어.', '0'], ['요즘 직장생활이 너무 편하고 좋은 것 같아!우리 회사는 복지가 정말 좋아! 내가 곧 출산하게 되는데 육아휴직 기간도 긴 

In [21]:
# Train & Test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=32)

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [22]:
# 데이터셋 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [23]:
# BERTSentenceTransform 클래스 정의
# BERT 모델에 입력으로 넣기 위한 전처리 과정
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset] # 문장 변환
    self.labels = [np.int32(i[label_idx]) for i in dataset] # label 변환

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self): ## 전체 데이터셋의 길이 변환
    return (len(self.labels))

In [24]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-6

In [25]:
# Data tokenization, int encoding, padding

# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
tok = tokenizer.tokenize # 토크나이저 가져옴

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [26]:
# torch 형식의 dataset을 만들어 입력 데이터셋 전처리 마무리
# 테스트를 수행하기 위한 데이터로 로드
train_dataloader = DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## KOBERT 모델 구현

In [27]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size=768,
               num_classes=6, # 감정 클래스 수로 조정
               dr_rate=0.1,
               params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifer = nn.Linear(hidden_size, num_classes)  # 선형 레이어를 통해 BERT의 출력을 감정 클래스로 매핑
    if dr_rate: # 드롭아웃 레이어를 선택적으로 추가
      self.dropout = nn.Dropout(p=dr_rate)
  # 어텐션 마스크 생성 : 패딩된 부분 가리기
  # 불필요하게 패딩 토큰에 대해서 어텐션하지 않기
  # 1 : 실제 단어이므로 마스킹 하지 않음
  # 0 : 패딩 토큰이므로 마스킹한다
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):  # 예측값 생성
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    # pooler: BERT 모델의 출력 중 하나로, 전체 시퀀스를 요약하는 역할
    _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device), return_dict=False)
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifer(out)


In [30]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer와 schedule 설정 (Linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimzier_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimzier_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices==Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

train_dataloader

In [32]:
for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0
  model.train() # 학습 모드로 설정
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):  # 학습 데이터셋을 순회하면서 각 배치에 대해 반복
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids) # 모델의 데이터를 전달하고 예측 얻기
    loss = loss_fn(out, label)  # 손실 계산, 역전파 수행
    loss.backward() # gradient explore 방지
    nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()  # 모델의 파라미터 업데이트
    scheduler.step()  # Update learning rate schedule
    train_acc += calc_accuracy(out, label)  # 정확도 누적
    if batch_id % log_interval == 0:
      print(f"epoch {e+1} batch id {batch_id+1} loss {loss.data.cpu().numpy()} train acc {train_acc/(batch_id+1)}")
  print(f"epoch {e+1} train acc {batch_id+1}")

  model.eval()  # 평가 모드로 설정
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)): # 테스트 데이셋을 순회하면서 각 배치에 대해 반복
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device) # 필요한 데이터를 GPU로 이동
    out = model(token_ids, valid_length, segment_ids) # 모델에 데이터를 전달하고 예측 얻음
    test_acc += calc_accuracy(out, label) # 정확도 누적
  print(f"epoch {e+1} test acc {test_acc/(batch_id+1)}")

<ipython-input-32-df2187b9e251>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):  # 학습 데이터셋을 순회하면서 각 배치에 대해 반복


  0%|          | 0/729 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9333323240280151 train acc 0.171875
epoch 1 batch id 201 loss 1.7601678371429443 train acc 0.16371268656716417
epoch 1 batch id 401 loss 1.5215098857879639 train acc 0.20285224438902744
epoch 1 batch id 601 loss 1.2316594123840332 train acc 0.3019966722129784
epoch 1 train acc 729


<ipython-input-32-df2187b9e251>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)): # 테스트 데이셋을 순회하면서 각 배치에 대해 반복


  0%|          | 0/183 [00:00<?, ?it/s]

...

epoch 4 test acc 0.6657030640124902


  0%|          | 0/729 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.9582526087760925 train acc 0.6875
epoch 5 batch id 201 loss 0.6184379458427429 train acc 0.689521144278607
epoch 5 batch id 401 loss 0.9400871396064758 train acc 0.6882793017456359
epoch 5 batch id 601 loss 1.0009046792984009 train acc 0.6874740016638935
epoch 5 train acc 729


  0%|          | 0/183 [00:00<?, ?it/s]

epoch 5 test acc 0.6655322989851679


## Predict 함수 정의


In [33]:
def predict(predict_sentence):  # input = 감정분류 하고자 하는 sentence
  data = [predict_sentence, '0']  # input 준비 및 전처리
  dataset_another = [data]  # predict sentence를 튜플로 변환

  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # BERT 모델에 이볅으로 사용할 데이터셋 생성 : 토큰화한 문장
  test_dataloader = DataLoader(another_test, batch_size=batch_size, num_workers=5)  # torch 형식 변환: mini batch

  model.eval()  # 모델 평가 모드

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length = valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids) # 모델에 미니배치를 입력으로 주어 예측값을 얻습니다.
    test_eval = []
    for i in out: # out = model(token_ids, valid_length, segment_ids)
      logits = i
      logits = logits.detach().cpu().numpy()

      # 감정 레이블 및 퍼센트 계산
      emotion_labels = ["불안", "분노", "상처", "슬픔", "당황", "기쁨"]
      probabilities = [np.exp(logit) / np.sum(np.exp(logits)) * 100 for logit in logits]

      for label, percent in zip(emotion_labels, probabilities):
        print(f"{label}: {percent:.2f}%")

      max_index = np.argmax(logits)
      max_emotion_label = emotion_labels[max_index]
      print(f">> 입력하신 내용에서 {max_emotion_label} 느껴집니다.")

In [38]:
# 질문에 0 입력시 종료
end = 1
while end == 1:
  sentence = input("하고 싶은 말을 입력해주세요 : ")
  if sentence == "0":
    break
  predict(sentence)
  print("\n")

하고 싶은 말을 입력해주세요 : 안녕하세요 잘부탁드립니다!!
불안: 1.45%
분노: 1.16%
상처: 0.67%
슬픔: 1.42%
당황: 0.61%
기쁨: 94.69%
>> 입력하신 내용에서 기쁨 느껴집니다.


하고 싶은 말을 입력해주세요 : 모두들 일년동안 열심히 해봅시다
불안: 0.87%
분노: 0.94%
상처: 0.53%
슬픔: 0.74%
당황: 0.39%
기쁨: 96.53%
>> 입력하신 내용에서 기쁨 느껴집니다.


하고 싶은 말을 입력해주세요 : 안녕하세요 조건희입니다.
불안: 6.67%
분노: 3.85%
상처: 2.64%
슬픔: 6.66%
당황: 2.70%
기쁨: 77.48%
>> 입력하신 내용에서 기쁨 느껴집니다.


하고 싶은 말을 입력해주세요 : 0


In [35]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 73.2 MB/s eta 0:00:00


In [37]:
import torch.onnx

# 더미 입력을 GPU로 이동
dummy_input = (
    torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]).to(device), # token_ids
    torch.LongTensor([10]).to(device),  # valid_length
    torch.LongTensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]).to(device),  # segment_ids
)

# 모델을 ONNX 형식으로 내보냅니다.
onnx_path = 'emotion_model.onnx'
torch.onnx.export(model, dummy_input, onnx_path, verbose=True, input_names=['input'], output_names=['output'])

from google.colab import files
files.download('emotion_model.onnx')

<ipython-input-27-e950220cafea>:21: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, v in enumerate(valid_length):


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>